In [ ]:

import serial
import time
import numpy as np
import matplotlib.pyplot as plt
def send_cmd(ser, cmd):
    """Send a command and wait a bit"""
    ser.write((cmd + '\r').encode())
    time.sleep(0.1)

def query(ser, cmd):
    """Send a command and read the response"""
    send_cmd(ser, cmd)
    return ser.readline().decode().strip()

# Open serial connection
ser = serial.Serial(
    port='COM7',         # Replace with your actual port
    baudrate=9600,
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=2,
    xonxoff=False,
    rtscts=False,
    dsrdtr=False
)

def init_cmds(ser):
    # Reset and configure instrument
    print(query(ser, '*IDN?'))
    send_cmd(ser, '*RST')
    send_cmd(ser, ':FORM:ELEM READ')
    send_cmd(ser, 'TRIG:DEL 0')
    send_cmd(ser, 'TRIG:COUNT 1')
    send_cmd(ser, 'SENS:CURR:NPLC 6')
    send_cmd(ser, 'SENS:CURR:RANG 0.000002')
    send_cmd(ser, 'SENS:CURR:RANG:AUTO ON')
    send_cmd(ser,"SYST:ZCOR ON")
    send_cmd(ser, 'SYST:AZER:STAT OFF')
    send_cmd(ser, 'DISP:ENAB ON')
    send_cmd(ser, ':SYST:ZCH:STAT OFF')

    # Wait for settings to take effect
    time.sleep(0.5)

init_cmds(ser)

#417 reads/min with no time.sleep
num_of_reads = 256
readings = np.zeros(num_of_reads)
dt = np.zeros(num_of_reads)
starttime = time.time()

# Trigger a read and fetch result
for i in range(num_of_reads):
    time.sleep(0.1)
    current = query(ser, 'READ?')
    readings[i] = current
    dt[i] = time.time() - starttime

# Optional: close serial port when done
ser.close()
plt.scatter(dt, readings*1e9)
plt.axhline(np.median(readings)*1e9, color='red', linestyle='--', label='Median Current')
plt.axhline(np.mean(readings)*1e9, color='green', linestyle='--', label='Mean Current')
plt.xlabel('Time (s)')
plt.ylabel('Current (nA)')
plt.show()


SerialException: could not open port 'COM7': PermissionError(13, 'Access is denied.', None, 5)